In [0]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!ls /content/drive/My\ Drive/colab

bkai00mp.ttf		dict.txt.big		titanic
dictionary.csv		Inverted_index.csv	title.json
dictionary__.json	jsonJieba-tran.json	userdict_oldd.txt
dictionary.json		pm2.5Taiwan.csv		userdict_old.txt
dictionary_noun_id.csv	Questions_200.json	userdict.txt
dictt.txt.big		questions_example.json	Wordcount.csv


In [0]:
!pip install ijson

     |████████████████████████████████| 61kB 4.6MB/s 


# ijson: https://github.com/isagalaev/ijson 使用ijson搜尋工具，ijson主要是使用在json過大而不能夠全部loading進記憶體時，採用stream的方式讀取json檔。

In [0]:
import ijson
f = open("/content/drive/My Drive/colab/jsonJieba-tran.json")
objects = ijson.items(f, 'item')
list_kw = []

for article in list(objects)[0:]:
  #print(article["id"]),
  #print(article["title"]),
  #print(article["content"])
  list_kw.append(str(article["id"]))
  list_kw.append(article["content"].split("/"))


In [0]:
print(len(list_kw))
#print(list_kw[1])

In [0]:
print(len(list_kw))
print(list_kw[0])
print(list_kw[1])
print(list_kw[2])
print(list_kw[3])

In [0]:
def StoredN(list_kw):
  list_dic = []

  for element in range(len(list_kw)):
    list_tmp = []
    if element%2 == 0:
      id = list_kw[element]
    elif element%2 == 1:
      for content in list_kw[element]:
        if content == '':
          continue
          print(list_kw[element])
        else:
          tmp = content.split(' ',1)
          if tmp[1][0] == 'n':
            list_tmp.append(tmp[0] + " " + id)
      list_dic.append(list_tmp)
  return list_dic
  
list_kw = StoredN(list_kw)



In [0]:
print(list_kw[0])
print(len(list_kw[0]))
print(list_kw[1])
print(len(list_kw[1]))

# SaveAs Dictionary.csv

In [0]:
import csv
fp = open("/content/drive/My Drive/colab/dictionary.csv", "w")
w = csv.writer(fp)

#w.writerow(['Noun','id'])
for doc in list_kw:
  for ele in doc:
    kk = ele.split(' ',1)
    w.writerow([kk[0],kk[1]])
  #w.writerow("\n")
fp.close()

# SaveAs Inverted_index.csv

In [0]:
!apt-get update

In [0]:
#環境初始化 (大約三至五分鐘)
! wget -O init_env.sh https://www.dropbox.com/s/6bnwn8u2hz19s59/init_env.sh && \
bash init_env.sh

<h3> Build RDD </h3>

In [0]:
import os, sys
os.environ['SPARK_HOME'] = "/usr/local/spark"
os.environ['PYSPARK_PYTHON'] = "/usr/local/bin/python"
sys.path.append("/usr/local/spark/python/")
sys.path.append("/usr/local/spark/python/lib/pyspark.zip")
sys.path.append("/usr/local/spark/python/lib/py4j-0.10.4-src.zip")

In [0]:
from pyspark import SparkContext
from pyspark import SparkConf
sc = SparkContext()

In [0]:
rdd = sc.textFile("/content/drive/My Drive/colab/dictionary.csv")

In [0]:
rdd.first()

In [0]:
#rdd.take(5)

In [0]:
rdd = rdd.map(lambda line: line.split(',',1))

In [0]:
def KeyValueGen(x):
  return (x[0], x[1])

In [0]:
reduced_rdd = rdd.map(KeyValueGen)

In [0]:
reduced_rdd.take(5)

In [0]:
test_rdd = reduced_rdd.reduceByKey(lambda x,y: x + "," + y)

In [0]:
test_rdd.take(1)

In [0]:
def DelDup(x):
  ID = x[1].split(",")
  first = []
  second = ""
  for id in ID:
    if id not in first:
      if len(first) == 0:
        first.append(id)
        second += id
        continue
      first.append(id)
      second += "," + id
  return (x[0], second)

In [0]:
final_rdd = test_rdd.map(DelDup)

In [0]:
final_rdd.take(1)

In [0]:
def toCSVLine(data):

  return ','.join(str(d) for d in data)

lines = final_rdd.map(toCSVLine)

lines.saveAsTextFile('/content/drive/My Drive/colab/Inverted_index.csv')

# SaveAs WordCount

In [0]:
def count(x):
  return (x[0], 1)

tmp_rdd = rdd.map(count)
happy_rdd = tmp_rdd.reduceByKey(lambda x,y: x+y)
happy_rdd1 = happy_rdd.map(lambda x: (x[0],str(x[1])))

In [0]:
import csv
fp = open("/content/drive/My Drive/colab/Wordcount.csv", "w")
w = csv.writer(fp)
test_list = happy_rdd1.collect()

print(test_list)
for doc in test_list:
  w.writerow(doc)
fp.close()

In [0]:
count_rdd = sc.textFile("/content/drive/My Drive/colab/Wordcount.csv")
count_rdd.take(10)